# DCT

In [8]:
import numpy as np
import pandas as pd
import cv2
from scipy.fftpack import dct
from pathlib import Path
from tqdm import tqdm


FILENAME = 'DlibDataChangeLuminace'

dbPath = '../' + FILENAME + '/DB/jpeg/'
queryPath = '../' + FILENAME + '/Query/jpeg/'
best_score = 0

if __name__ == '__main__':
    #with open("dct.csv", mode='w') as f:
    dbList = []
    queryList = []
    distanceList = []
    answerList = []
    count = 0
    # print('c1: ' + str(i) + ', c2: ' + str(j))
    #f.write('query_name,answer\n')
    p = Path(dbPath)
    p = sorted(p.glob("*.jpg"))
    feature_name_list = []
    for i in range(225):
        featrue_name = 'dct_' + str(i)
        feature_name_list.append(featrue_name)
    db_df = pd.DataFrame(columns=feature_name_list)
    for index, filename in enumerate(tqdm(p)):
        img = cv2.imread(dbPath + filename.name, 0)
        # cv2.imwrite(DBDctOriginalSavePath + filename.name, cv2.dct(np.float32(img)))
        img = cv2.GaussianBlur(img, (7, 7), 2)
        img = cv2.Canny(img, 12, 34)
        # img = cv2.Canny(img, c1, c2)
        # cv2.imwrite(DBCannySavePath + filename.name, img)
        imf = np.float32(img)
        dst = cv2.dct(imf)
        dst = dst[:15,:15]
        for ind, name in enumerate(feature_name_list):
            db_df.loc[index, name] = np.array(dst).flatten()[ind]
        # cv2.imwrite(DBDctSavePath + filename.name, dst)
        dbList.append(np.float32(dst))
    p = Path(queryPath)
    p = sorted(p.glob("*.jpg"))
    query_df = pd.DataFrame(columns=feature_name_list)
    for index, filename in enumerate(tqdm(p)):
        distanceList = []
        img = cv2.imread(queryPath + filename.name, 0)
        # cv2.imwrite(QueryDctOriginalSavePath + filename.name, cv2.dct(np.float32(img)))
        img = cv2.GaussianBlur(img, (7, 7), 2)
        img = cv2.Canny(img, 12, 34)
        # img = cv2.Canny(img, c1, c2)
        # cv2.imwrite(QueryCannySavePath + filename.name, img)
        imf = np.float32(img)
        dst = cv2.dct(imf)
        dst = dst[:15,:15]
        for ind, name in enumerate(feature_name_list):
            query_df.loc[index, name] = np.array(dst).flatten()[ind]
        # cv2.imwrite(QueryDctSavePath + filename.name, dst)
        queryList.append(np.float32(dst))
        for index in range(len(dbList)):
            distance  = (dbList[index][:15, :15] - queryList[-1][:15, :15])**2
            distanceList.append(distance.sum())
        distSort = np.argsort(distanceList)
        answerList.append(np.uint8(np.argmin(distanceList)/10))
        ranking = np.uint8(distSort[:10]/10)
        mode = np.argmax(np.bincount(ranking))
        #print(str(ranking) + '   ' + str(answerList[-1]))
        #print('mode: ' + str(mode))
        # if mode != answerList[-1]:
        target = filename.name[0:2]
        if target[0] == 'r':
            target = 0
        else:
            target = int(target)
        distanceList = []
        #f.write(filename.name + ',' + str(answerList[-1]) + '\n')
    if db_df.min().min() < query_df.min().min():
        norm_min = db_df.min().min()
    else:
        norm_min = query_df.min().min()

    db_df = db_df - norm_min
    query_df = query_df - norm_min

    if db_df.max().max() > query_df.max().max():
        norm_max = db_df.max().max()
    else:
        norm_max = query_df.max().max()

    db_df = db_df / norm_max
    query_df = query_df / norm_max

    p = Path(dbPath)
    p = sorted(p.glob("*.jpg"))
    for index, filename in enumerate(tqdm(p)):
        db_df.loc[index, 'target'] = int(index/10)
    db_df.to_csv("db_list.csv")

    p = Path(queryPath)
    p = sorted(p.glob("*.jpg"))
    for index, filename in enumerate(tqdm(p)):
        target = filename.name[0:2]
        if target[0] == 'r':
            target = 0
        else:
            target = int(target)
        query_df.loc[index, 'target'] = target
    query_df.to_csv("query_list.csv")

    accuracy  = count / len(queryList) * 100
    print(str(accuracy) + '%')

100%|████████████████████████████████████████████████████████████████████████████████| 58/58 [00:00<00:00, 1362.43it/s]


0.0%


In [9]:
db_df.head()

,dct_0,dct_1,dct_2,dct_3,dct_4,dct_5,dct_6,dct_7,dct_8,dct_9,...,dct_216,dct_217,dct_218,dct_219,dct_220,dct_221,dct_222,dct_223,dct_224,target
0,0.737262,0.175016,0.113924,0.153127,0.13369,0.176395,0.154656,0.170932,0.107212,0.125349,...,0.1411,0.146259,0.137682,0.129752,0.149151,0.1349,0.153167,0.163225,0.153085,0.0
1,0.751443,0.141763,0.0898414,0.137095,0.160792,0.145021,0.141301,0.114819,0.148148,0.149846,...,0.159639,0.147005,0.123542,0.152843,0.171788,0.16318,0.143531,0.160291,0.150175,0.0
2,0.749183,0.182122,0.0642995,0.151828,0.146128,0.167398,0.159621,0.188366,0.106396,0.116415,...,0.135148,0.153333,0.162617,0.167818,0.15068,0.115388,0.163913,0.139544,0.145554,0.0
3,0.75391,0.188737,0.0741698,0.135332,0.136272,0.174824,0.157801,0.164014,0.104855,0.125693,...,0.157162,0.139359,0.141461,0.120198,0.169114,0.149013,0.138937,0.162534,0.146803,0.0
4,0.749388,0.139903,0.0765032,0.130673,0.167958,0.155368,0.165349,0.124899,0.121281,0.144036,...,0.15014,0.155673,0.157584,0.147971,0.156413,0.175099,0.106058,0.157175,0.15254,0.0


In [10]:
query_df.head()

,dct_0,dct_1,dct_2,dct_3,dct_4,dct_5,dct_6,dct_7,dct_8,dct_9,...,dct_216,dct_217,dct_218,dct_219,dct_220,dct_221,dct_222,dct_223,dct_224,target
0,0.886156,0.123158,0.0722077,0.0943952,0.140728,0.125841,0.140765,0.125505,0.100931,0.13188,...,0.161136,0.146858,0.160145,0.145711,0.165835,0.153856,0.152171,0.158962,0.144106,0.0
1,0.68588,0.132094,0.105261,0.120549,0.0905193,0.119517,0.0890462,0.127166,0.113726,0.159056,...,0.149095,0.137558,0.178444,0.142637,0.161745,0.158365,0.161033,0.150266,0.140211,0.0
2,0.892641,0.144309,0.0718136,0.0880258,0.112929,0.119989,0.161151,0.147221,0.188399,0.141104,...,0.17683,0.140744,0.146447,0.164933,0.155706,0.143431,0.129375,0.124857,0.145472,0.0
3,0.904602,0.125462,0.0756135,0.103461,0.136243,0.131062,0.169615,0.149847,0.156416,0.133878,...,0.165096,0.152867,0.158161,0.153803,0.151534,0.158782,0.139724,0.156603,0.139849,0.0
4,0.753704,0.159946,0.110662,0.125843,0.0878895,0.16897,0.144036,0.138922,0.16487,0.144578,...,0.15165,0.124137,0.136217,0.14999,0.16187,0.174851,0.150873,0.179958,0.162893,0.0


In [12]:
print(db_df.max().max())
print(db_df.min().min())
print(query_df.max().max())
print(query_df.min().min())

19.0
0.0
19.0
0.0
